In [1]:
import numpy as np
import tensorflow as tf
from basenji import seqnn
import json
from basenji import params

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
with open('cs273b_params.json') as params_open:
    params = json.load(params_open)
params_model = params['model']
params_train = params['train']

# g = tf.Graph()
# with g.as_default():
# Define operations and tensors in `g`.
model = seqnn.SeqNN(params_model)

In [ ]:
#model.restore('cs273b_model.h5', trunk=True)
model.model.load_weights('cs273b_model.h5')

In [ ]:
print(model.model.layers[-1].output)
output = model.model.layers[-1].output
s1 = -1*tf.keras.backend.sum(output, axis=2)
s2 = 2*output[:,0]
print(s1)
print(s2)
s3 = tf.keras.backend.concatenate((s1, s2), axis=1)
print(s3)
s4 = tf.keras.backend.sum(s3, axis=1)
print(s4)
s5 = tf.keras.backend.mean(s4)
print(s5)
#s2 = tf.keras.backend.add(s, 2*output[:,0,0])
#print(s2)

In [21]:
tf.compat.v1.disable_eager_execution()
output_class = 2 # the index of the output class we want to maximize

g = tf.Graph()
with g.as_default():
    output = model.model.layers[-1].output
    output = tf.keras.backend.reshape(output, shape=(-1, 4))

    print(output)
    s1 = -1*tf.keras.backend.sum(output, axis=1)
    print(s1)
    s2 = 2*output[:,output_class]
    print(s2)
    s3 = tf.keras.backend.concatenate((s1, s2), axis=0)
    print(s3)
    s4 = tf.keras.backend.sum(s3, axis=0)
    print(s4)
    # loss = tf.keras.backend.mean(s4)
    #loss = tf.keras.backend.mean(output[:,output_class])
    #loss = output
    loss = s4
    print(loss)
    
    print(model.model.input)
    grads = tf.compat.v2.gradients(loss, model.model.input)[0] # the output of `gradients` is a list, just take the first (and only) element
    print("jre")
    grads = tf.keras.backend.l2_normalize(grads) # normalize the gradients to help having an smooth optimization process
    print(grads)

    iterate = tf.keras.backend.function([model.model.input], [loss, grads])


Tensor("Reshape_15:0", shape=(None, 4), dtype=float32)
Tensor("mul_12:0", shape=(None,), dtype=float32)
Tensor("mul_13:0", shape=(None,), dtype=float32)
Tensor("concat_4:0", shape=(None,), dtype=float32)
Tensor("Sum_10:0", shape=(), dtype=float32)
Tensor("Sum_10:0", shape=(), dtype=float32)
Tensor("sequence:0", shape=(None, 1000, 4), dtype=float32)


ValueError: Node 'gatherv2_params' expects to be colocated with unknown node 'stochastic_shift/cond'

In [ ]:
#input_img = np.random.random((1, 28, 28)) # define an initial random image

lr = 1.  # learning rate used for gradient updates
max_iter = 50  # number of gradient updates iterations
for i in range(max_iter):
    loss_val, grads_val = iterate([input_img])
    input_img += grads_val * lr  # update the image based on gradients
    
pred = model.predict(input_img)
print(pred)
    

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [10]:
model_t = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=1)

In [ ]:
model.summary()
print(dict([(layer.name, layer) for layer in model.layers]))

In [ ]:
from keras.preprocessing.image import save_img
from keras import layers
from keras.applications import vgg16
from keras import backend as K

In [11]:
#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()
print(tf.executing_eagerly())
output_class = 2 # the index of the output class we want to maximize
output = model_t.layers[-1].output
print(output)
loss = tf.keras.backend.mean(output[:,output_class]) # get the average activation of our desired class over the batch
print(loss)
print(model_t.input)
x = 1/0
grads = tf.compat.v2.gradients(loss, model.input)[0] # the output of `gradients` is a list, just take the first (and only) element
grads = tf.keras.backend.l2_normalize(grads) # normalize the gradients to help having an smooth optimization process
print(grads)
# loss = tf.keras.backend.mean(model_output[1])
# # # compute the gradient of the input picture wrt this loss

# grads = tf.compat.v2.gradients(loss, input_img)[0]

# # # normalization trick: we normalize the gradient
# # grads /= (tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(grads))) + 1e-5)
iterate = tf.keras.backend.function([model.input], [loss, grads])


False
Tensor("dense_2/Softmax:0", shape=(None, 10), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)
Tensor("flatten_input:0", shape=(None, 28, 28), dtype=float32)


ZeroDivisionError: division by zero

In [ ]:
input_img = np.random.random((1, 28, 28)) # define an initial random image

lr = 1.  # learning rate used for gradient updates
max_iter = 50  # number of gradient updates iterations
for i in range(max_iter):
    loss_val, grads_val = iterate([input_img])
    input_img += grads_val * lr  # update the image based on gradients
    
pred = model.predict(input_img)
print(pred)
    

In [ ]:
x = input_img
x -= x.mean()
x /= (x.std() + tf.keras.backend.epsilon())
x *= 0.25

# clip to [0, 1]
x += 0.5
x = np.clip(x, 0, 1)

# convert to RGB array
x *= 255
if tf.keras.backend.image_data_format() == 'channels_first':
    x = x.transpose((1, 2, 0))
x = np.clip(x, 0, 255).astype('uint8')

In [ ]:
import matplotlib.pyplot as plt
img = np.reshape(x, (x.shape[1], x.shape[2]))
plt.imshow(img)